In [2]:
from pathlib import Path
from transformers import pipeline
import networkx as nx
import matplotlib.pyplot as plt
from nltk.tokenize import sent_tokenize
import nltk
nltk.download("punkt_tab")

C:\Users\sayfe\Desktop\PolytechS9_2\web_semantique\WebSem_movie_project\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\sayfe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [23]:
# Load the triplet extractor
triplet_extractor = pipeline('text2text-generation', model='Babelscape/rebel-large', tokenizer='Babelscape/rebel-large')
print("triplet extractor loaded")

All PyTorch model weights were used when initializing TFBartForConditionalGeneration.

All the weights of TFBartForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.
Device set to use 0


triplet extractor loaded


In [19]:
#path to text
input_file = Path("../data/oscars_text.txt")
# Read and preprocess text
input_text = input_file.read_text()
# Tokenize into sentences
sentences = sent_tokenize(input_text)
print(sentences)

['ï»¿Kevin Spacey and Thora Birch appear in a scene from American Beauty.', 'The film won the Oscar for Best Picture at the 2000 Academy Awards.', 'Kevin Spacey also received the Best Actor Oscar for his role as Lester, and the film won three additional Oscars for Best Director (Sam Mendes), Best Original Screenplay (Alan Ball), and Best Cinematography (Conrad L. Hall).', 'In 2001, Gladiator tells the story of Roman General Maximus, played by Russell Crowe, who becomes a gladiator to avenge the murder of his family.', 'This film won the Oscar for Best Picture at the 2001 ceremony.', 'Russell Crowe earned the Best Actor Oscar for his outstanding performance, and the film also won Oscars for Best Costume Design, Best Sound, and Best Visual Effects.', 'The biographical drama A Beautiful Mind, released in 2002, follows mathematical genius John Forbes Nash Jr., played by Russell Crowe, who struggles with schizophrenia while developing a loving relationship with Alicia Larde, portrayed by Je

In [20]:
# Group sentences into chunks of 3
# nb_sentences_per_chunk = 3
# chunks = [' '.join(sentences[i:i+nb_sentences_per_chunk]) for i in range(0, len(sentences), nb_sentences_per_chunk)]
# with open("chunks.txt", "w") as f:
#     for chunk in chunks:
#         f.write(chunk + "\n\n")
# len(chunks)

# Read text from oscars_text.txt create chunks at each line break
with open("../data/oscars_text.txt", "r") as f:
    text = f.read()
    chunks = text.split("\n\n")
    print(chunks)


['ï»¿Kevin Spacey and Thora Birch appear in a scene from American Beauty. The film won the Oscar for Best Picture at the 2000 Academy Awards. Kevin Spacey also received the Best Actor Oscar for his role as Lester, and the film won three additional Oscars for Best Director (Sam Mendes), Best Original Screenplay (Alan Ball), and Best Cinematography (Conrad L. Hall).', 'In 2001, Gladiator tells the story of Roman General Maximus, played by Russell Crowe, who becomes a gladiator to avenge the murder of his family. This film won the Oscar for Best Picture at the 2001 ceremony. Russell Crowe earned the Best Actor Oscar for his outstanding performance, and the film also won Oscars for Best Costume Design, Best Sound, and Best Visual Effects.', 'The biographical drama A Beautiful Mind, released in 2002, follows mathematical genius John Forbes Nash Jr., played by Russell Crowe, who struggles with schizophrenia while developing a loving relationship with Alicia Larde, portrayed by Jennifer Conne

In [21]:
with open("../data_result/chunks.txt", "w") as f:
    for chunk in chunks:
        f.write(chunk+"\n")
len(chunks)

20

In [22]:
# install tensorflow 2.12.0
# !pip install tf-keras

In [24]:
relations = set()

In [25]:
# Function to parse the generated text and extract the triplets
def extract_triplets(text):
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
        relations.add(relation.strip())
    if subject != '' and relation != '' and object_ != '':
        triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
    return triplets

In [26]:
# Iterate over each chunk, extract triplets and create directed knowledge graphs
# bigG = nx.DiGraph()
nb_chunks = len(chunks)
for i,chunk in enumerate(chunks):
    # print("Chunk number : ", i+1, "/" , nb_chunks , " has sentences number : ", i*nb_sentences_per_chunk+1, " to", i*nb_sentences_per_chunk+nb_sentences_per_chunk)
    extracted_text = triplet_extractor.tokenizer.batch_decode([triplet_extractor(chunk, return_tensors=True, return_text=False)[0]["generated_token_ids"]])
    extracted_triplets = extract_triplets(extracted_text[0])
    with open("../data_result/triplets.txt", "a") as f:
        f.write(str(extracted_triplets))
    print(chunk)
    # # Create directed graph with the triplets for the chunk
    # G = nx.DiGraph()
    # for triplet in extracted_triplets:
    #     G.add_edge(triplet['head'], triplet['tail'], type=triplet['type'])
    #     bigG.add_edge(triplet['head'], triplet['tail'], type=triplet['type'])
    #
    # # Increase node spacing and adjust layout
    # pos = nx.spring_layout(G, k=2, seed=42)  # Adjust k for spacing
    #
    # # Draw nodes and edges
    # plt.figure(figsize=(12, 10))  # Enlarge the figure
    # nx.draw_networkx_nodes(G, pos, node_color='lightblue', node_size=1500)
    # nx.draw_networkx_edges(
    #     G, pos, edge_color='gray', width=1.5, arrows=True, arrowsize=15, node_size=1500
    # )
    # # Draw labels with bounding boxes for better visibility
    # nx.draw_networkx_labels(
    #     G, pos, font_size=10, font_color='black', font_weight='bold',
    #     bbox=dict(facecolor='white', edgecolor='none', alpha=0.8)
    # )
    #
    # # Edge labels
    # edge_labels = nx.get_edge_attributes(G, 'type')
    # nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8, rotate=True)
    #
    # # Add margins and optimize layout
    # plt.margins(0.3)  # Add space around the graph
    # plt.axis('off')  # Hide axes
    # plt.tight_layout()
    # plt.show()


ï»¿Kevin Spacey and Thora Birch appear in a scene from American Beauty. The film won the Oscar for Best Picture at the 2000 Academy Awards. Kevin Spacey also received the Best Actor Oscar for his role as Lester, and the film won three additional Oscars for Best Director (Sam Mendes), Best Original Screenplay (Alan Ball), and Best Cinematography (Conrad L. Hall).
In 2001, Gladiator tells the story of Roman General Maximus, played by Russell Crowe, who becomes a gladiator to avenge the murder of his family. This film won the Oscar for Best Picture at the 2001 ceremony. Russell Crowe earned the Best Actor Oscar for his outstanding performance, and the film also won Oscars for Best Costume Design, Best Sound, and Best Visual Effects.
The biographical drama A Beautiful Mind, released in 2002, follows mathematical genius John Forbes Nash Jr., played by Russell Crowe, who struggles with schizophrenia while developing a loving relationship with Alicia Larde, portrayed by Jennifer Connelly. The

In [31]:
# Write the relations to a file
with open("../data_result/relations.txt", "w") as f:
    for relation in relations:
        f.write(relation + "\n")

In [32]:
# Define file paths
input_file = "../data_result/triplets.txt"
output_file = "../data_result/good_triplets.csv"

# Read the input file
with open(input_file, "r") as file:
    content = file.read()

# Perform the replacements
processed_content = (
    content.replace(",  ", "")  # Replace '},' with a newline
        .replace(", ", "")    # Replace ', ' with a newline
        .replace("[", "")    # Remove '['
        .replace("]", "")    # Remove ']'
        .replace("}", "\n")    # Remove '['
        .replace("{", "")    # Remove ']'
        .replace("''", "','")    # Remove '['
        .replace("'", "")    # Remove '['
        .replace("head: ", "")    # Remove '['
        .replace("type: ", "")    # Remove '['
        .replace("tail: ", "")    # Remove '['
)

# Write the processed content to the output file
with open(output_file, "w") as file:
    file.write("head,type,tail\n")
    file.write(processed_content)

print(f"Processing complete. Output written to {output_file}.")

Processing complete. Output written to ../data_result/good_triplets.csv.


In [35]:
# Split csv into multiple files for each "type"
import pandas as pd
df = pd.read_csv("../data_result/good_triplets.csv")
types = ["award received", "director", "cast member"]
for type_ in types:
    df[df["type"] == type_][["head","tail"]].rename(columns={"head": "movie_entity", "tail": type_}).to_csv(f"../data_result/new_{type_}.csv", index=False)

In [30]:
# # Visualize the full text knowledge graph
#
# # Increase node spacing and adjust layout
# pos = nx.spring_layout(bigG, k=2, seed=42)  # Adjust k for spacing
#
# # Draw nodes and edges
# plt.figure(figsize=(12, 10))  # Enlarge the figure
# nx.draw_networkx_nodes(bigG, pos, node_color='lightblue', node_size=1500)
# nx.draw_networkx_edges(
#     bigG, pos, edge_color='gray', width=1.5, arrows=True, arrowsize=15, node_size=1500
# )
# # Draw labels with bounding boxes for better visibility
# nx.draw_networkx_labels(
#     bigG, pos, font_size=10, font_color='black', font_weight='bold',
#     bbox=dict(facecolor='white', edgecolor='none', alpha=0.8)
# )
#
# # Edge labels
# edge_labels = nx.get_edge_attributes(bigG, 'type')
# nx.draw_networkx_edge_labels(bigG, pos, edge_labels=edge_labels, font_size=8, rotate=True)
#
# # Add margins and optimize layout
# plt.margins(0.3)  # Add space around the graph
# plt.axis('off')  # Hide axes
# plt.tight_layout()
# plt.show()
# # save figure
# plt.savefig("output.png")